<a href="https://colab.research.google.com/github/romanozamoth/transfer-learning-ml-dio/blob/main/New_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers torch torchvision scikit-learn pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━ 16.0/24.6 MB 58.9 MB/s eta 0:00:01ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [6]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os
import numpy as np
from sklearn.linear_model import LogisticRegression
import joblib


In [ ]:
from zipfile import ZipFile
from google.colab import files

# Subir o seu dataset.zip com os modelos que quer treinar
uploaded = files.upload()

# Supondo que você fez upload de dataset.zip com estrutura de pastas
with ZipFile("dataset.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")


In [ ]:
# Carrega modelo CLIP
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cpu")

def get_clip_features(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to("cpu")
    with torch.no_grad():
        features = model.get_image_features(**inputs)
    return features[0].numpy()

X, y = [], []
labels = ["cachorro", "pato", "passaro", "peixe"]
base_path = "dataset/"

for label in labels:
    for file in os.listdir(f"{base_path}/{label}"):
        path = os.path.join(base_path, label, file)
        if path.lower().endswith((".png", ".jpg", ".jpeg")):
            feat = get_clip_features(path)
            X.append(feat)
            y.append(label)

clf = LogisticRegression(max_iter=1000)
clf.fit(X, y)


In [7]:
joblib.dump(clf, "modelo_clip.pkl")


NameError: name 'clf' is not defined

In [4]:
from google.colab import files
files.download("modelo_clip.pkl")


FileNotFoundError: Cannot find file: modelo_clip.pkl

In [3]:
import joblib
clf = joblib.load("modelo_clip.pkl")


FileNotFoundError: [Errno 2] No such file or directory: 'modelo_clip.pkl'

In [ ]:
# TESTAR MODELO
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt

# Função para prever e exibir resultado
def prever_imagem(path_imagem):
    # Abrir e exibir imagem
    img = Image.open(path_imagem).convert("RGB")

    # Extrair embeddings e fazer predição
    features = get_clip_features(path_imagem)  # Essa função deve estar definida antes
    predicao = clf.predict([features])[0]

    # Exibir imagem com título da predição
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predição: {predicao}")
    plt.show()

# Fazer upload de imagem
uploaded = files.upload()

# Para cada imagem enviada, mostrar predição
for fn in uploaded.keys():
    prever_imagem(fn)
